In [1]:
!pwd

/home/programmer/Desktop/general/End-To-End-MLOps-Project/notebooks


In [2]:
import os

os.chdir("../")
!pwd

/home/programmer/Desktop/general/End-To-End-MLOps-Project


In [3]:
from src.utils.common import read_yaml
from src.constants import PARAMS_FILE_PATH

import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib


import warnings
warnings.filterwarnings("ignore")

In [27]:
config = read_yaml(PARAMS_FILE_PATH)

remote_server_uri = config.mlflow_config.remote_server_uri
mlflow.set_tracking_uri(remote_server_uri)


runs = mlflow.search_runs(experiment_ids=[0,1,2])
runs


[2023-07-09 13:05:38,829: INFO: common: yaml file: params.yaml loaded successfully]


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.acu,metrics.f1_score,params.criterion,params.max_depth,params.min_samples_split,params.min_samples_leaf,params.n_estimators,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.git.commit,tags.mlflow.user,tags.mlflow.log-model.history
0,58b5411185604bb7a0102cb00e0c4b01,2,FINISHED,/home/programmer/Desktop/general/End-To-End-ML...,2023-07-09 07:31:35.946000+00:00,2023-07-09 07:32:07.174000+00:00,0.990226,0.990311,log_loss,30,2,1,200,LOCAL,src/pipeline/stage_03_training_and_evaluation.py,mlops,8c710c53cc6b75374903698186b147a6603fa1c9,programmer,"[{""run_id"": ""58b5411185604bb7a0102cb00e0c4b01""..."
1,c8d49c6ceb70445f80c3d1a025d095a1,2,FINISHED,/home/programmer/Desktop/general/End-To-End-ML...,2023-07-09 07:24:32.165000+00:00,2023-07-09 07:25:41.558000+00:00,0.979424,0.979818,log_loss,20,4,2,10,LOCAL,/home/programmer/.local/lib/python3.10/site-pa...,mlops,None,programmer,"[{""run_id"": ""c8d49c6ceb70445f80c3d1a025d095a1""..."
2,ea9aed84b6224dc982071a8131dfdf78,2,FINISHED,/home/programmer/Desktop/general/End-To-End-ML...,2023-07-08 13:08:31.309000+00:00,2023-07-08 13:09:52.801000+00:00,0.991255,0.991322,gini,30,4,1,100,LOCAL,src/pipeline/stage_03_training_and_evaluation.py,mlops,700237ae2e33edcc3056ddc5b91a096704c28d82,programmer,"[{""run_id"": ""ea9aed84b6224dc982071a8131dfdf78""..."


In [31]:
runs["metrics.acu"].sort_values(ascending=False, ignore_index=True)

0    0.991255
1    0.990226
2    0.979424
Name: metrics.acu, dtype: float64

In [32]:
highest = runs["metrics.acu"].sort_values(ascending=False, ignore_index=True)[0]
highest

0.9912551440329218

In [33]:
runs[runs["metrics.acu"]==highest]

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.acu,metrics.f1_score,params.criterion,params.max_depth,params.min_samples_split,params.min_samples_leaf,params.n_estimators,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.source.git.commit,tags.mlflow.user,tags.mlflow.log-model.history
2,ea9aed84b6224dc982071a8131dfdf78,2,FINISHED,/home/programmer/Desktop/general/End-To-End-ML...,2023-07-08 13:08:31.309000+00:00,2023-07-08 13:09:52.801000+00:00,0.991255,0.991322,gini,30,4,1,100,LOCAL,src/pipeline/stage_03_training_and_evaluation.py,mlops,700237ae2e33edcc3056ddc5b91a096704c28d82,programmer,"[{""run_id"": ""ea9aed84b6224dc982071a8131dfdf78""..."


In [38]:
runs[runs["metrics.acu"]==highest]['run_id']

2    ea9aed84b6224dc982071a8131dfdf78
Name: run_id, dtype: object

In [42]:
highest_run_id = runs[runs["metrics.acu"]==highest]["run_id"]
highest_run_id = highest_run_id.reset_index(drop=True)[0]
highest_run_id

'ea9aed84b6224dc982071a8131dfdf78'

In [43]:
model_name = config.mlflow_config.registered_model_name

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)

{   'aliases': [],
    'creation_timestamp': 1688887721974,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1688887721974,
    'name': 'Random_Forest_Register',
    'run_id': 'ea9aed84b6224dc982071a8131dfdf78',
    'run_link': '',
    'source': '/home/programmer/Desktop/general/End-To-End-MLOps-Project/artifacts/2/ea9aed84b6224dc982071a8131dfdf78/artifacts/models_mlflow',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


In [44]:
model_name = config.mlflow_config.registered_model_name

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Production"
        )
    else:
        current_version = mv["version"]
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Staging"
        )    

{   'aliases': [],
    'creation_timestamp': 1688887721974,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1688887721974,
    'name': 'Random_Forest_Register',
    'run_id': 'ea9aed84b6224dc982071a8131dfdf78',
    'run_link': '',
    'source': '/home/programmer/Desktop/general/End-To-End-MLOps-Project/artifacts/2/ea9aed84b6224dc982071a8131dfdf78/artifacts/models_mlflow',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


In [45]:
loaded_model = mlflow.pyfunc.load_model(logged_model)

model_path = config.webapp_model_dir
joblib.dump(loaded_model, model_path)

2023/07/09 13:16:52 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.1.1, required: mlflow<3,>=2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


['saved_models/model.joblib']